Template for displaying waveforms
-----

by davide.franco@apc.in2p3.fr

In [ ]:
import sys,os, re
sys.path.insert(0, '../')

# Load environment
if re.search('cnaf',os.uname()[1]):  
    print('Loading CNAF environment')
    sys.path.append('/opt/exp_software/darkside/software/python_local_libraries/')
    sys.path.append('/cvmfs/sft.cern.ch/lcg/views/LCG_96python3/x86_64-centos7-gcc8-opt/lib/python3.6/site-packages/')
    sys.path.append('/cvmfs/sft.cern.ch/lcg/views/LCG_96python3/x86_64-centos7-gcc8-opt/lib')


In [ ]:
from pyreco.manager.manager import Manager
#use view option only with small amount of data
#cmdline_args = "--config laser.ini --output output_file --view"
filename = '/storage/gpfs_ds50/darkside/naples2021tests/run00235sub005.mid.lz4'
cmdline_args = f'--config pdm_reco.ini --output output_file -i {filename}'
m = Manager(cmdline_args=cmdline_args)

In [ ]:
from pyreco.event.midas import MIDAS
import matplotlib.pyplot as plt 
import numpy as np 
lista = {'input': m.config.input}
events = MIDAS(lista)
event_number = 100
for n,ev in enumerate(events):
    if n < event_number: continue 
    if n > event_number: break
    wfs = m.algos.get_subtracted_waveform(ev, gate=500)
    wfs = wfs[0:-1,:]
    for i,wf in enumerate(wfs):
        plt.plot(wf, label=f'{i}')

plt.xlim(3500,4500)
#plt.legend()
plt.show()

In [ ]:
from pyreco.reco.filtering import WFFilter
mfilter = WFFilter(m.config)
flt = np.reshape(mfilter.numba_fast_filter(wfs),newshape=wfs.shape)
for i,wf in enumerate(flt):
        plt.plot(wf, label=f'{i}')

plt.xlim(3500,4500)
plt.show()

In [ ]:
mas = m.algos.running_mean(flt,gate=500)
flt = flt - mas 
#flt = np.where(flt>0,flt, 0)
for i,wf in enumerate(flt):
    rms = m.algos.get_rms(wf)    
    plt.plot(np.where(wf > 3*rms, wf, 0), label=f'{i}')

plt.xlim(3500,4500)
plt.show()